**Kalman FIlter Code for GGE6102**

This code is designed to import a CSV file containing one of the follwoing:
* UTM X, with Velocity East (m/s) and Acceleration East (m/s^2)
* UTM Y, with Velocity North (m/s) and Acceleration North (m/s^2)
* Height, with Velocity Up (m/s) and Acceleration Up (m/s^2)

This code will then compute the prediction step using the velocity and acceleration given. It will then input the next UTM coord and calculate the correction. This will then be uploaded to a CSV file.

In [11]:
import numpy as np

import matplotlib as mpl


Here we will define the function to import the CSV and manipulate the variables within

In [12]:
import csv

#function defini
# tion

def row_data(filename):
  mylist = []
  with open(filename) as example:
    example_data = csv.reader(example, delimiter=',')
    next(example_data)

    for row in example_data:
      mylist.append(row)

    return mylist

new_list = row_data(r'C:\Users\Avery\OneDrive\Documents\GitHub\GGE6102\Data_Sept_F180_Combined.csv')

# it is important to note that no additional rows were added to the file

Now that the CSV is imported into a list, further reduction is needed to make lists for all the different variables

In [13]:
#defining time as a list to bring the elements from the time column
time1 = []
for row in new_list:
  time1.append(row[0:1])

# Here I am removing the [] from each element
flat_time = []
for sublist in time1:
  for item in sublist:
    flat_time.append(item)

#the function eval() removes the '' from each element
T = [eval(i) for i in flat_time]

#------------------------------------------------------------------------------

#defining X as a list to bring the elements from the UTM X coord column
X1 = []
for row in new_list:
  X1.append(row[3:4])

flat_X = []
for sublist in X1:
  for item in sublist:
    flat_X.append(item)

X = [eval(i) for i in flat_X]

#------------------------------------------------------------------------------

#defining Y as a list to bring the elements from the UTM Y coord column
Y1 = []
for row in new_list:
  Y1.append(row[4:5])

flat_Y = []
for sublist in Y1:
  for item in sublist:
    flat_Y.append(item)

Y = [eval(i) for i in flat_Y]

#------------------------------------------------------------------------------

#defining H as a list to bring the elements from the Height coord column
H1 = []
for row in new_list:
  H1.append(row[5:6])

flat_H = []
for sublist in H1:
  for item in sublist:
    flat_H.append(item)

H = [eval(i) for i in flat_H]

#------------------------------------------------------------------------------

#defining Vn as a list of elements from the Velocity North column
Vn1 = []
for row in new_list:
  Vn1.append(row[10:11])

flat_Vn = []
for sublist in Vn1:
  for item in sublist:
    flat_Vn.append(item)

Vn = [eval(i) for i in flat_Vn]

#------------------------------------------------------------------------------

#defining Ve as a list of elements from the Velocity East column
Ve1 = []
for row in new_list:
  Ve1.append(row[12:13])

flat_Ve = []
for sublist in Ve1:
  for item in sublist:
    flat_Ve.append(item)

Ve = [eval(i) for i in flat_Ve]

#------------------------------------------------------------------------------

#defining Vu as a list of elements from the Velocity Up column
Vu1 = []
for row in new_list:
  Vu1.append(row[14:15])

flat_Vu = []
for sublist in Vu1:
  for item in sublist:
    flat_Vu.append(item)

Vu = [eval(i) for i in flat_Vu]

#------------------------------------------------------------------------------

#defining stx as a list of elements from the Standard Deviation X column
stx1 = []
for row in new_list:
  stx1.append(row[7:8])

flat_stx = []
for sublist in stx1:
  for item in sublist:
    flat_stx.append(item)

stx = [eval(i) for i in flat_stx]

#------------------------------------------------------------------------------

#defining sty as a list of elements from the Standard Deviation Y column
sty1 = []
for row in new_list:
  sty1.append(row[6:7])

flat_sty = []
for sublist in sty1:
  for item in sublist:
    flat_sty.append(item)

sty = [eval(i) for i in flat_sty]

#------------------------------------------------------------------------------

#defining stu as a list of elements from the Standard Deviation Up column
stu1 = []
for row in new_list:
  stu1.append(row[8:9])

flat_stu = []
for sublist in stu1:
  for item in sublist:
    flat_stu.append(item)

stu = [eval(i) for i in flat_stu]

print(H[0:10])



[-16.2781, -16.2921, -16.3845, -16.6144, -16.7938, -17.0019, -17.2269, -17.4081, -17.2958, -16.9244]


Here I define the generic matrices and valuables

It is improtant to remeber that the change in time is one second as that is what the Septentrio and the F180 used

In [14]:
F = np.array([[1, 1],[0,1]])

G = np.array([[0],[1]])

HM = np.array([[1],[0]])

error_v = 0.014**2

error_acc = 0.014**2

Q = np.array([[error_acc, 0],[0, error_acc]])




Here I will calculate the Kalman filtering along the X axis


In [15]:
#Nomenclature: name_state; name: cov = covariance, 
#state: corr => corrected, pred => prediction

k_max = len(X) - 1
k_total = range(0,k_max)

X_position_corr = 0
Y_position_corr = 0
H_position_corr = 0

Final_X = []
Final_Y = []
Final_H = []

for k in k_total:
  k_next = int(k) + 1

  #Here I am using the if statement to define the initial situation
  if k == 0:
    X_cov_corr = np.array([[stx[k]**2, 0],[0,stx[k]**2]])
    Y_cov_corr = np.array([[sty[k]**2, 0],[0,sty[k]**2]])
    H_cov_corr = np.array([[stu[k]**2, 0],[0,stu[k]**2]])

  else:
    X_cov_corr = X_LHS * X_cov_pred
    Y_cov_corr = Y_LHS * Y_cov_pred
    H_cov_corr = H_LHS * H_cov_pred

  X_k_position = float(X[k])
  X_vel1 = float(Ve[k])
  X_vel2 = float(Ve[k_next])
  X_acc_k = X_vel2 - X_vel1

  Y_k_position = float(Y[k])
  Y_vel1 = float(Vn[k])
  Y_vel2 = float(Vn[k_next])
  Y_acc_k = Y_vel2 - Y_vel1
  
  H_k_position = float(H[k])
  H_vel1 = float(Vu[k])
  H_vel2 = float(Vu[k_next])
  H_acc_k = H_vel2 - H_vel1

  #Here I am defining the position observation from GNSS
  X_position_obs = X_k_position + stx[k]
  Y_position_obs = Y_k_position + sty[k]
  H_position_obs = H_k_position + stu[k]
    
  #The following two equations are the predictions
  X_position_pred = np.array([[X_k_position + X_vel1],[X_vel1 + X_acc_k]])
  Y_position_pred = np.array([[Y_k_position + Y_vel1],[Y_vel1 + Y_acc_k]])
  H_position_pred = np.array([[H_k_position + H_vel1],[H_vel1 + H_acc_k]])
  
  X_cov_pred = F * X_cov_corr * F.transpose() + Q
  Y_cov_pred = F * Y_cov_corr * F.transpose() + Q
  H_cov_pred = F * H_cov_corr * F.transpose() + Q
  
  #Correction step

  #The following is for the optimal gain
  X_denom = 3 * stx[k]**2 + 0.000392
  X_posi_up = 2 * stx[k]**2 + 0.000392
  X_opti_posi = float(X_posi_up / X_denom)
  X_opti_vel = 0.014**2 / X_denom
  X_optimal_gain = np.array([[X_opti_posi],[X_opti_vel]])
  X_position_corr = X[k] + Ve[k] + X_opti_posi
  Final_X.append(X_position_corr)
  X_LHS = np.array([[1-X_opti_posi, 0],[-1*X_opti_vel, 0]])


  Y_denom = 3 * sty[k]**2 + 0.000392
  Y_posi_up = 2 * sty[k]**2 + 0.000392
  Y_opti_posi = float(Y_posi_up / Y_denom)
  Y_opti_vel = 0.00196 / Y_denom
  Y_optimal_gain = np.array([[Y_opti_posi],[Y_opti_vel]])
  Y_position_corr = Y[k] + Vn[k] + Y_opti_posi
  Final_Y.append(Y_position_corr)
  Y_LHS = np.array([[1-Y_opti_posi, 0],[-1*Y_opti_vel, 0]])

  H_denom = 3 * stu[k]**2 + 0.000392
  H_posi_up = 2 * stu[k]**2 + 0.000392
  H_opti_posi = float(H_posi_up / H_denom)
  H_opti_vel = 0.00196 / H_denom
  H_optimal_gain = np.array([[H_opti_posi],[H_opti_vel]])
  H_position_corr = H[k] + Vu[k] + H_opti_posi
  Final_H.append(H_position_corr)
  H_LHS = np.array([[1-H_opti_posi, 0],[-1*H_opti_vel, 0]])

  



In [16]:
print(Final_X)
print(Final_Y)
print(Final_H)

[683883.0022484359, 683882.9838039915, 683882.9207262148, 683882.9950928813, 683883.1007039926, 683883.1325595488, 683883.1618039934, 683883.2429706609, 683883.2566817721, 683883.1726484396, 683883.1023706621, 683883.0731706623, 683883.0392817737, 683883.066615107, 683883.0754817745, 683883.1770039969, 683883.3205262204, 683883.3675262209, 683883.2950928872, 683883.2229928875, 683883.222581777, 683883.2383817774, 683883.2203706667, 683883.3163484451, 683883.2101595565, 683883.2068040009, 683883.2943484464, 683883.3021262246, 683883.3065040022, 683883.2146040025, 683883.1784262258, 683883.2504706704, 683883.1421484486, 683883.0385706711, 683883.0964595606, 683883.1800373389, 683883.2104595613, 683883.1762373395, 683883.1502373401, 683883.1521595626, 683883.1810484518, 683883.2031595632, 683883.1940928965, 683883.1599262309, 683883.0989595643, 683883.0175484536, 683883.0611373433, 683882.9888040094, 683882.9399928988, 683883.1091595663, 683883.1364484555, 683883.127259567, 683883.1876040

In [22]:
import pandas as pd

df = pd.DataFrame(data=Final_X)
df['Y'] = Final_Y
df['H'] = Final_H

df.to_csv('results1.csv')


df


,0,Y,H
0,683883.002248,5.092465e+06,-15.601711
1,683882.983804,5.092465e+06,-15.615711
2,683882.920726,5.092465e+06,-15.708111
3,683882.995093,5.092465e+06,-15.938011
4,683883.100704,5.092465e+06,-16.117411
...,...,...,...
3440,683883.358673,5.092463e+06,-16.184787
3441,683883.413117,5.092463e+06,-16.269254
3442,683883.566039,5.092463e+06,-16.259698
3443,683883.636161,5.092463e+06,-16.301776


In [23]:
import numpy as np

# Set the length of the state vector
n = len(X)

# Initialize the corrected state estimate and covariance
X_corr = np.zeros((n, 1))
P_corr = P

# Initialize the output variables
path = np.zeros((n, 2))

# Define the unscented transform parameters
alpha = 0.001
beta = 2
kappa = 0

# Define the weights for the sigma points
lam = alpha**2 * (n + kappa) - n
wm = np.full(2*n+1, 1/(2*(n+lam)))
wm[0] = lam/(n+lam)
wc = wm.copy()
wc[0] = wc[0] + (1 - alpha**2 + beta)

# Define the sigma point generation function
def sigma_points(x, P):
    L = np.linalg.cholesky((n+lam)*P)
    return np.hstack((x, np.tile(x, (n, 1)) + L, np.tile(x, (n, 1)) - L))

# Define the process model function
def f(x):
    # Extract the position and velocity states
    pos = x[:2]
    vel = x[2:]
    
    # Compute the velocity and acceleration estimates
    vel_est = vel + dt*a
    acc_est = (vel_est - vel)/dt
    
    # Concatenate the position and velocity estimates
    x_est = np.hstack((pos + dt*vel, vel_est))
    
    return x_est

# Define the measurement model function
def h(x):
    # Extract the position states
    pos = x[:2]
    
    # Compute the position estimate
    pos_est = pos
    
    return pos_est

# Loop over each time step
for k in range(n):
    # Generate the sigma points
    X_sigma = sigma_points(X_corr[:, k], P_corr[:, :, k])
    
    # Propagate the sigma points through the process model
    X_pred_sigma = np.apply_along_axis(f, 1, X_sigma)
    
    # Compute the predicted state estimate and covariance
    X_pred = np.sum(wm*X_pred_sigma, axis=0)
    P_pred = np.zeros((n, n))
    for i in range(2*n+1):
        diff = X_pred_sigma[:, i:i+1] - X_pred.reshape(-1, 1)
        P_pred = P_pred + wc[i]*np.dot(diff, diff.T)
    
    # Compute the sigma points for the predicted state estimate and covariance
    X_pred_sigma = sigma_points(X_pred, P_pred)
    
    # Propagate the sigma points through the measurement model
    Y_pred_sigma = np.apply_along_axis(h, 1, X_pred_sigma)
    
    # Compute the predicted observation estimate and covariance
    Y_pred = np.sum(wm*Y_pred_sigma, axis=0)
    Pyy = np.zeros((2, 2))
    Pxy = np.zeros((n, 2))
    for i in range(2*n+1):
        diff_y = Y_pred_sigma[:, i:i+1] - Y_pred.reshape(-1, 1)
        diff_x = X_pred_sigma[:, i:i+1] - X_pred.reshape(-1, 1)
        Pyy = Pyy + wc[i]*np.dot(diff_y, diff_y.T)

NameError: name 'P' is not defined